# Import 311 CARE/CARE+ Requests
* Aggregate by request type
* Spatial join with census tracts
* Count the number of requests for each request type by tract-year
* Move to script when ready

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
from shapely.geometry import Point
import boto3

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')
bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'

s3 = boto3.client('s3')

In [3]:
df = pd.read_csv(f'{bucket_name}data/raw/MyLA311_Service_Requests.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Create geometry column from lat/long
df['geometry'] = df.dropna(subset=['Latitude', 'Longitude']).apply(
    lambda x: Point(x.Longitude, x.Latitude), axis = 1)

In [5]:
df = gpd.GeoDataFrame(df)
df.crs = {'init':'epsg:4326'}

df = df[df.geometry.notna()]
df = df.to_crs({'init':'epsg:2229'})

## Extract information needed on map

In [6]:
df.RequestType.value_counts()

Homeless Encampment            65505
Illegal Dumping Pickup         16606
Bulky Items                    12071
Service Not Complete             222
Metal/Household Appliances       151
Electronic Waste                  61
Dead Animal Removal                3
Illegal Dumping in Progress        1
Name: RequestType, dtype: int64

In [7]:
# Define a function that will aggregate Request Type
def request_type(row):
    
    homeless = 0
    bulky = 0
    illegal = 0
    other = 0
    
    if row.RequestType == 'Homeless Encampment':
        homeless = 1
    elif (row.RequestType == 'Bulky Items'):
        bulky = 1
    elif 'Illegal Dumping' in row.RequestType:
        illegal = 1
    elif ((row.RequestType == 'Service Not Complete') or 
          (row.RequestType == 'Metal/Household Appliances') or 
          (row.RequestType == 'Electronic Waste') or 
          (row.RequestType == 'Dead Animal Removal')):
        other = 1
    
    return pd.Series([homeless, bulky, illegal, other], index=['homeless', 'bulky', 'illegal', 'other'])

In [8]:
requests = df.apply(request_type, axis = 1)
df = pd.concat([df, requests], axis = 1)

In [9]:
# Extract year 
for col in ['CreatedDate', 'UpdatedDate']:
    df[col] = pd.to_datetime(df[col])

df['year'] = df.CreatedDate.dt.year

In [10]:
df.year.value_counts()

2019    34114
2018    30247
2017    19935
2016     9067
2015     1257
Name: year, dtype: int64

In [11]:
df.head()

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,CDMember,NC,NCName,PolicePrecinct,geometry,homeless,bulky,illegal,other,year
0,1-1487677271,2019-11-06 13:17:37,2019-11-06 13:47:38,SR Created,BOS,Homeless Encampment,Pending,Self Service,Self Service,NaN,...,Nury Martinez,112.0,NORTH HILLS EAST,MISSION,POINT (6420904.738 1904547.785),1,0,0,0,2019
1,1-1487079981,2019-11-05 23:02:38,2019-11-06 21:23:23,SR Created,BOS,Illegal Dumping Pickup,Pending,Call,BOS,NaN,...,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL,POINT (6484760.638 1836564.385),0,0,1,0,2019
2,1-1487079951,2019-11-05 23:01:17,2019-11-06 22:41:56,SR Created,BOS,Bulky Items,Pending,Call,BOS,NaN,...,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL,POINT (6484760.638 1836564.385),0,1,0,0,2019
3,1-1487080711,2019-11-05 22:58:10,2019-11-06 22:41:44,SR Created,BOS,Bulky Items,Pending,Call,BOS,NaN,...,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL,POINT (6484760.638 1836564.385),0,1,0,0,2019
4,1-1487051388,2019-11-05 21:21:10,2019-11-06 22:17:54,SR Created,BOS,Homeless Encampment,Pending,Mobile App,Self Service,iOS,...,Paul Krekorian,25.0,NC VALLEY VILLAGE,NORTH HOLLYWOOD,POINT (6441738.081 1878829.038),1,0,0,0,2019


## Spatial join with census tracts

In [12]:
tracts = gpd.read_file(f'{bucket_name}gis/raw/census_tracts.geojson')

In [13]:
m1 = gpd.sjoin(df, tracts, how = 'inner', op = 'intersects')

In [14]:
m1.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'CreatedByUserOrganization',
       'MobileOS', 'Anonymous', 'AssignTo', 'ServiceDate', 'ClosedDate',
       'DateServiceRendered', 'ReasonCode', 'ResolutionCode',
       'AddressVerified', 'ApproximateAddress', 'Address', 'HouseNumber',
       'Direction', 'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude',
       'Location', 'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember',
       'NC', 'NCName', 'PolicePrecinct', 'geometry', 'homeless', 'bulky',
       'illegal', 'other', 'year', 'index_right', 'GEOID'],
      dtype='object')

In [15]:
pivot1 = m1.pivot_table(index = ['GEOID', 'year'], 
               values = ['homeless', 'bulky', 'illegal', 'other'], aggfunc = 'sum').reset_index().sort_values(['GEOID', 'year'])

# Pivot wouldn't work with a geometry column. 
# Merge geometry column for tracts back in
pivot1 = pd.merge(pivot1, tracts, on = 'GEOID', validate = 'm:1')

pivot1 = gpd.GeoDataFrame(pivot1)
pivot1.crs = {'init':'epsg:2229'}

In [16]:
# Export to S3
pivot1.to_file(driver = 'GeoJSON', filename = '../gis/care311_tracts.geojson')

s3.upload_file('../gis/care311_tracts.geojson', 'city-of-los-angeles-data-lake', 
               'public-health-dashboard/gis/intermediate/care311_tracts.geojson')